<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Image_Segmentaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import sys
import skimage.io
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

tf.__version__

'2.9.2'

In [2]:
!git clone https://github.com/ahmedfgad/Mask-RCNN-TF2.git

Cloning into 'Mask-RCNN-TF2'...
remote: Enumerating objects: 1440, done.
remote: Total 1440 (delta 0), reused 0 (delta 0), pack-reused 1440
Receiving objects: 100% (1440/1440), 157.00 MiB | 28.96 MiB/s, done.
Resolving deltas: 100% (794/794), done.


In [4]:
%cd Mask-RCNN-TF2

/content/Mask-RCNN-TF2


In [ ]:
!python setup.py install

In [6]:
ROOT_DIR = os.path.abspath('./Mask-RCNN-TF2')
ROOT_DIR

'/content/Mask-RCNN-TF2/Mask-RCNN-TF2'

In [7]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.8/dist-packages/IPython/extensions',
 '/root/.ipython']

In [8]:
sys.path.append(ROOT_DIR)
sys.path

['/content',
 '/env/python',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.8/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/Mask-RCNN-TF2/Mask-RCNN-TF2']

In [14]:
%cd /content/Mask-RCNN-TF2/

/content/Mask-RCNN-TF2


In [23]:
from mrcnn import utils
from mrcnn import model
from mrcnn import config
from mrcnn import visualize

In [24]:
sys.path.append(os.path.join(ROOT_DIR, 'samples/coco/'))

In [27]:
%cd /content/Mask-RCNN-TF2/samples
import coco
%cd /content/Mask-RCNN-TF2/

/content/Mask-RCNN-TF2


In [28]:
MODEL_DIR = os.path.join(ROOT_DIR, 'logs')
IMAGE_DIR = os.path.join(ROOT_DIR, 'images')